In [8]:
import pandas as pd
import numpy as np
from sklearn import *

import matplotlib.pyplot as plt

%matplotlib inline

In [5]:
df = pd.read_csv("/data/kaggle/data_combined_cleaned.csv")
del df["Id"]
df = df.dropna()
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalesPrice
0,60,RL,65.0,8450,Pave,None,Reg,Lvl,Inside,Gtl,...,0,None,None,None,0,2,2008,WD,Normal,208500.0
1,20,RL,80.0,9600,Pave,None,Reg,Lvl,FR2,Gtl,...,0,None,None,None,0,5,2007,WD,Normal,181500.0
2,60,RL,68.0,11250,Pave,None,IR1,Lvl,Inside,Gtl,...,0,None,None,None,0,9,2008,WD,Normal,223500.0
3,70,RL,60.0,9550,Pave,None,IR1,Lvl,Corner,Gtl,...,0,None,None,None,0,2,2006,WD,Abnorml,140000.0
4,60,RL,84.0,14260,Pave,None,IR1,Lvl,FR2,Gtl,...,0,None,None,None,0,12,2008,WD,Normal,250000.0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 0 to 1459
Data columns (total 79 columns):
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            1460 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-null object
Exterior2nd      1460 non-null object
MasVnrType       1460 no

In [9]:
target = "SalesPrice"

X = df.drop(columns=target)
X_dummy = pd.get_dummies(X, drop_first=True)

y = np.log(df[target])

X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X_dummy, y , test_size = 0.3, random_state =1)

pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.LinearRegression())
])


pipe.fit(X_train, y_train)

y_test_pred = pipe.predict(X_test)
y_train_pred = pipe.predict(X_train)

print("r2 training: ", metrics.r2_score(y_train, y_train_pred))
print("r2 test: ", metrics.r2_score(y_test, y_test_pred))

print("rmse train: ", metrics.mean_squared_error(y_train, y_train_pred) ** 0.5)
print("rmse test: ", metrics.mean_squared_error(y_test, y_test_pred)** 0.5)


r2 training:  0.9494518297088844
r2 test:  -3.0715943247509893e+22
rmse train:  0.08606817747710854
rmse test:  76190488351.17424


In [21]:
est = pipe.steps[-1][-1]
thetas = pd.DataFrame({"feature": X_dummy.columns
                       , "coeff":  est.coef_})
thetas["abs_coeff"] = np.abs(thetas["coeff"])
thetas.sort_values("abs_coeff", ascending=False)

,feature,coeff,abs_coeff
15,GrLivArea,1.501695e+11,1.501695e+11
13,2ndFlrSF,-1.233520e+11,1.233520e+11
12,1stFlrSF,-1.112376e+11,1.112376e+11
219,GarageFinish_None,-1.072428e+11,1.072428e+11
134,Exterior2nd_CBlock,8.844505e+10,8.844505e+10
...,...,...,...
249,SaleType_ConLw,-3.542900e-04,3.542900e-04
45,LotShape_Reg,-2.273321e-04,2.273321e-04
158,Foundation_Slab,-1.544952e-04,1.544952e-04
42,Alley_Pave,-2.288818e-05,2.288818e-05


In [19]:
thetas.columns

Index(['feature', 'coeff '], dtype='object')

In [24]:
m = pd.DataFrame({"actual": y_test, "predicted": y_test_pred})
m.sort_values("predicted", ascending = False)

,actual,predicted
1270,12.468437,1.281787e+12
375,11.018629,9.484743e+11
1003,11.827043,1.510345e+09
1011,11.512925,1.066149e+09
120,12.100712,5.829678e+08
...,...,...
968,10.542706,1.100410e+01
916,10.471950,1.089222e+01
1000,11.314475,1.086910e+01
636,11.002100,1.080757e+01


In [28]:
target = "SalesPrice"

X = df.drop(columns=target)
X_dummy = pd.get_dummies(X, drop_first=True)

y = np.log(df[target])

X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X_dummy, y , test_size = 0.3, random_state =1)

pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.Lasso(alpha=0.01))
])


pipe.fit(X_train, y_train)

y_test_pred = pipe.predict(X_test)
y_train_pred = pipe.predict(X_train)

print("r2 training: ", metrics.r2_score(y_train, y_train_pred))
print("r2 test: ", metrics.r2_score(y_test, y_test_pred))

print("rmse train: ", metrics.mean_squared_error(y_train, y_train_pred) ** 0.5)
print("rmse test: ", metrics.mean_squared_error(y_test, y_test_pred)** 0.5)

est = pipe.steps[-1][-1]
thetas = pd.DataFrame({"feature": X_dummy.columns
                       , "coeff":  est.coef_})
thetas["abs_coeff"] = np.abs(thetas["coeff"])
thetas.sort_values("abs_coeff", ascending=False)



r2 training:  0.8940480673803641
r2 test:  0.8785389987905549
rmse train:  0.12460754170603186
rmse test:  0.15150859029885974


,feature,coeff,abs_coeff
3,OverallQual,0.104135,0.104135
15,GrLivArea,0.095563,0.095563
25,GarageCars,0.055896,0.055896
5,YearBuilt,0.029465,0.029465
39,MSZoning_RM,-0.026432,0.026432
...,...,...,...
109,RoofStyle_Shed,0.000000,0.000000
110,RoofMatl_CompShg,0.000000,0.000000
111,RoofMatl_Membran,0.000000,0.000000
112,RoofMatl_Metal,0.000000,0.000000
